# Data Treatment for integrations of peaks in experiments



In [1]:
import pandas as pd
import numpy as np
import os
!pip freeze > requirements.txt



In [2]:
# Function to convert text time format to hours
def text_to_hours(text):
    if text.endswith('m'):
        return float(text[:-1]) / 60
    elif text.endswith('h'):
        return float(text[:-1])
    elif text.endswith('d'):
        return float(text[:-1]) * 24
    else:
        raise ValueError("Unknown time format: " + text)

def filter_txt_files(file_list):
    """Return only files ending with .txt from the input list."""
    return [f for f in file_list if f.endswith('.txt')]

In [3]:
results_FLS = []
# Read all txt files in the 'FLS' directory
list_datafile_FLS = filter_txt_files(os.listdir(os.path.join('.','FLS')))

for i in list_datafile_FLS:
    df = pd.read_csv(os.path.join('.','FLS',i), sep=',',header=20)
    integral = np.trapezoid(df[(df['Detector'] >= 500) & (df['Detector'] <= 525)]['red pmt'],
                            df[(df['Detector'] >= 500) & (df['Detector'] <= 525)]['Detector'])
    list_txt = i.split('.')[0].split('_')
    number_of_mix = list_txt[-3]
    number_of_batch = list_txt[-2]
    time = text_to_hours(list_txt[-1])
    C = (integral - 151763.70779)/(1.60489*10**11)
    # Placeholder values for V, D, Q, %, % cumulative
    V = None
    D = None
    Q = None
    percent = None
    percent_cumulative = None

    results_FLS.append({
        "Number of mix": number_of_mix,
        "Number of Batch": number_of_batch,
        "Time (h)": time,
        "V": V,
        "D": D,
        "A": integral,
        "C": C,
        "Q": Q,
        "%": percent,
        "% cumulative": percent_cumulative
    })

df_results_FLS = pd.DataFrame(results_FLS)
df_results_FLS

,Number of mix,Number of Batch,Time (h),V,D,A,C,Q,%,% cumulative
0,2,1,0.5,None,None,2.052074e+07,0.000127,None,None,None
1,2,1,1.0,None,None,2.052074e+07,0.000127,None,None,None


In [4]:
results_UV = []
# Read all txt files in the 'UV-vis' directory
list_datafile_UV = filter_txt_files(os.listdir(os.path.join('.','UV-vis')))
#
for i in list_datafile_UV:
    df = pd.read_csv(os.path.join('.','UV-vis',i),sep="\t",header=1)
    A = df[(df['Wavelength nm.'] == 490)]['Abs.'].values[0]
    list_txt = i.split('.')[0].split('_')
    number_of_mix = list_txt[-3]
    number_of_batch = list_txt[-2]
    time = text_to_hours(list_txt[-1])
    C = (A - 0.01555)/(48458.1651)
    # Placeholder values for V, D, Q, %, % cumulative
    V = None
    D = None
    Q = None
    percent = None
    percent_cumulative = None

    results_UV.append({
        "Number of mix": number_of_mix,
        "Number of Batch": number_of_batch,
        "Time (h)": time,
        "V": V,
        "D": D,
        "A": A,
        "C": C,
        "Q": Q,
        "%": percent,
        "% cumulative": percent_cumulative
    })

df_results_UV = pd.DataFrame(results_UV)
df_results_UV

,Number of mix,Number of Batch,Time (h),V,D,A,C,Q,%,% cumulative
0,2,1,0.5,None,None,1.3248,0.000027,None,None,None


In [5]:
#sort values by 'Number of Batch', 'Number of mix', and 'Time (h)'

df_results_FLS = df_results_FLS.sort_values(by=['Number of Batch', 'Number of mix', 'Time (h)'])
df_results_UV = df_results_UV.sort_values(by=['Number of Batch', 'Number of mix', 'Time (h)'])

In [6]:
# Export data to excel
df_results_FLS.to_excel('results_FLS.xlsx', index=False)
df_results_UV.to_excel('results_UV.xlsx', index=False)